In [1]:
import cv2
import time
import numpy as np
import mediapipe as mp
import math
from keras.models import load_model

from gtts import gTTS
import os
from playsound import playsound

import arabic_reshaper
from bidi.algorithm import get_display
from PIL import ImageFont, ImageDraw, Image

In [2]:
# Data
alpha = []
# Arabic data
arabic_words =['أنا', 'أنت', 'ارقام', 'اسرة', 'اسم', 'الأحد', 'الأربعاء',
       'الاثنين', 'الثلاثاء', 'الجمعة', 'الحمد لله', 'الخميس', 'السبت',
       'السلام عليكم ورحمة الله وبركاته', 'النهاردة', 'امرأة',
       'تاريخ ميلاد', 'جامعة', 'حروف', 'رجل', 'رقم التليفون', 'ساكن فين',
       'سلام', 'سنة', 'طالب في', 'عامل ايه', 'عندك كام اخوة',
       'عندك كام سنة', 'كل سنة وانت طيب', 'كلية الهندسة', 'كويس',
       'ممكن نتعارف', 'ولد']
arabic_words = np.array(arabic_words)

arabic_alphabet = ['ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا', 'ارقام', 'ال', 'ب', 'ة', 'ت', 'ث',
       'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ',
       'ع', 'غ', 'ف', 'ق', 'ك', 'كلمات', 'ل', 'لا', 'م', 'مسافة', 'ن',
       'ه', 'و', 'ى', 'ي']
arabic_alphabet = np.array(arabic_alphabet)

arabic_numbers = ['حروف', 'كلمات', '٠', '١', '١٠', '١١', '١٢', '١٣', '١٤', '١٥',
       '١٦', '١٧', '١٨', '١٩', '٢', '٢٠', '٢٣', '٣', '٤', '٥', '٦', '٧',
       '٨', '٩']
arabic_numbers = np.array(arabic_numbers)

# English data
english_words =['alphabet', 'faculty of engineering', 'fine', 'goodbye', 'hello',
       'how are you', 'is', 'my', 'name', 'numbers', 'student at',
       'university', 'what', 'your']
english_words = np.array(english_words)

english_alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'numbers', 'o', 'p', 'q', 'r', 's', 'space', 't', 'u', 'v',
       'w', 'words', 'x', 'y', 'z']
english_alphabet = np.array(english_alphabet)

# english_numbers = []
# english_numbers = np.array(english_numbers)

# Load the Models
arabic_words_model = load_model('Arabic_Data_Words.h5')
arabic_alphabet_model = load_model('Arabic_Data_Alphabet.h5')
arabic_numbers_model = load_model('Arabic_Data_Numbers.h5')

english_words_model = load_model('English_Data_Words.h5')
english_alphabet_model = load_model('English_Data_Alphabet.h5')
# english_numbers_model = load_model('English_Data_Numbers.h5')

model = arabic_words_model
actions = arabic_words

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

def mediapipe_detection(frame, model):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # Model (Holistic) works on RGB not BGR
    frame.flags.writeable = False # To improve performance, optionally mark the image as not writeable to pass by reference
    results = model.process(frame)                 # Make prediction
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return frame, results

def draw_landmarks(image, results):
    # Right hand
    mp_drawing.draw_landmarks(
        image, 
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Left Hand
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    )

In [4]:
def extract_keypoints(results):
    if results.right_hand_landmarks and results.left_hand_landmarks:
        # Right hand
        rightHand = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.right_hand_landmarks.landmark]).flatten()) if results.right_hand_landmarks else list(np.zeros(21*4))
        # Left hand
        leftHand = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.left_hand_landmarks.landmark]).flatten()) if results.left_hand_landmarks else list(np.zeros(21*4))
    
    elif results.right_hand_landmarks:
        # Right hand
        rightHand = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.right_hand_landmarks.landmark]).flatten()) if results.right_hand_landmarks else list(np.zeros(21*4))
        # Left hand
        leftHand = list(np.zeros(21*4))
        
    elif results.left_hand_landmarks:
        # Right hand
        rightHand = list(np.zeros(21*4))
        # Left hand
        leftHand = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in results.left_hand_landmarks.landmark]).flatten()) if results.left_hand_landmarks else list(np.zeros(21*4))
     
    else:
        rightHand = leftHand = list(np.zeros(21*4))
    return rightHand + leftHand

In [5]:
def getDistanceFromWrist(frame, results):
    h, w, c = frame.shape
    
    if results.right_hand_landmarks and results.left_hand_landmarks:
        thumbRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[4].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[4].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        indexRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[8].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[8].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        middleRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[12].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[12].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        ringRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[16].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[16].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        pinkyRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[20].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[20].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        
        thumbLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[4].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[4].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        indexLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[8].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[8].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        middleLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[12].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[12].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        ringLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[16].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[16].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        pinkyLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[20].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[20].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        
    elif results.right_hand_landmarks:
        thumbRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[4].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[4].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        indexRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[8].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[8].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        middleRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[12].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[12].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        ringRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[16].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[16].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        pinkyRight_wrist = float(math.sqrt((results.right_hand_landmarks.landmark[20].y * h - results.right_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[20].x * w - results.right_hand_landmarks.landmark[0].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        thumbLeft_wrist = indexLeft_wrist = middleLeft_wrist = ringLeft_wrist = pinkyLeft_wrist = 0.0
        
        
    elif results.left_hand_landmarks:
        thumbLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[4].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[4].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        indexLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[8].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[8].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        middleLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[12].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[12].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        ringLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[16].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[16].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        pinkyLeft_wrist = float(math.sqrt((results.left_hand_landmarks.landmark[20].y * h - results.left_hand_landmarks.landmark[0].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[20].x * w - results.left_hand_landmarks.landmark[0].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        thumbRight_wrist = indexRight_wrist = middleRight_wrist = ringRight_wrist = pinkyRight_wrist = 0.0
    
    else:
        thumbRight_wrist = indexRight_wrist = middleRight_wrist = ringRight_wrist = pinkyRight_wrist = thumbLeft_wrist = indexLeft_wrist = middleLeft_wrist = ringLeft_wrist = pinkyLeft_wrist = 0.0
        
    return thumbRight_wrist, indexRight_wrist, middleRight_wrist, ringRight_wrist, pinkyRight_wrist, thumbLeft_wrist, indexLeft_wrist, middleLeft_wrist, ringLeft_wrist, pinkyLeft_wrist

In [6]:
def getDistanceFromEar(frame, results):
    h, w, c = frame.shape
    if results.right_hand_landmarks and results.left_hand_landmarks:
        thumbRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[4].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[4].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        indexRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[8].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[8].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        middleRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[12].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[12].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        ringRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[16].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[16].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        pinkyRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[20].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[20].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0

        thumbLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[4].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[4].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        indexLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[8].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[8].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        middleLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[12].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[12].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        ringLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[16].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[16].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        pinkyLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[20].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[20].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
               
    elif results.right_hand_landmarks:      
        thumbRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[4].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[4].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        indexRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[8].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[8].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        middleRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[12].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[12].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        ringRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[16].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[16].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        pinkyRight_ear = float(math.sqrt((results.right_hand_landmarks.landmark[20].y * h - results.pose_landmarks.landmark[8].y * h)**2 + 
                                 (results.right_hand_landmarks.landmark[20].x * w - results.pose_landmarks.landmark[8].x * w)**2))\
                    if results.right_hand_landmarks else 0.0
        thumbLeft_ear = indexLeft_ear = middleLeft_ear = ringLeft_ear = pinkyLeft_ear = 0.0

    elif results.left_hand_landmarks:      
        thumbLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[4].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[4].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        indexLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[8].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[8].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        middleLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[12].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[12].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        ringLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[16].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[16].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        pinkyLeft_ear = float(math.sqrt((results.left_hand_landmarks.landmark[20].y * h - results.pose_landmarks.landmark[7].y * h)**2 + 
                                 (results.left_hand_landmarks.landmark[20].x * w - results.pose_landmarks.landmark[7].x * w)**2))\
                    if results.left_hand_landmarks else 0.0
        thumbRight_ear = indexRight_ear = middleRight_ear = ringRight_ear = pinkyRight_ear = 0.0
        
    else:
        thumbRight_ear = indexRight_ear = middleRight_ear = ringRight_ear = pinkyRight_ear = thumbLeft_ear = indexLeft_ear = middleLeft_ear = ringLeft_ear = pinkyLeft_ear = 0.0
        
    return thumbRight_ear, indexRight_ear, middleRight_ear, ringRight_ear, pinkyRight_ear, thumbLeft_ear, indexLeft_ear, middleLeft_ear, ringLeft_ear, pinkyLeft_ear

In [7]:
def clac_slope(a, wrist):
    x = 0
    y = 1
    if -1.0 < (a[x] - wrist[x]) < 0.0:
        return (a[y] - wrist[y]) / float(math.floor(a[x] - wrist[x]))
    elif 1.0 > (a[x] - wrist[x]) > 0.0:
        return (a[y] - wrist[y]) / float(math.ceil(a[x] - wrist[x]))
    elif (a[x] - wrist[x]) == 0.0:
        return (a[y] - wrist[y]) / float(a[x] - wrist[x] + 0.01)
    else:
        return (a[y] - wrist[y]) / float(a[x] - wrist[x])

def tan_angle(m1, m2):
    if (1 + m1 * m2) == 0.0:
        return (m1 - m2) / (1 + m1 * m2 +0.01)
    else:
        return (m1 - m2) / (1 + m1 * m2)

def clac_angle_Right():
    if results.right_hand_landmarks:
        wrist  = (int(results.right_hand_landmarks.landmark[0].x * w), int(results.right_hand_landmarks.landmark[0].y * h))
        thumb  = (int(results.right_hand_landmarks.landmark[4].x * w), int(results.right_hand_landmarks.landmark[4].y * h))
        index  = (int(results.right_hand_landmarks.landmark[8].x * w), int(results.right_hand_landmarks.landmark[8].y * h))
        middle = (int(results.right_hand_landmarks.landmark[12].x * w), int(results.right_hand_landmarks.landmark[12].y * h))
        ring   = (int(results.right_hand_landmarks.landmark[16].x * w), int(results.right_hand_landmarks.landmark[16].y * h))
        pinky  = (int(results.right_hand_landmarks.landmark[20].x * w), int(results.right_hand_landmarks.landmark[20].y * h))

        m_thumb  = clac_slope(thumb, wrist)
        m_index  = clac_slope(index, wrist)
        m_middle = clac_slope(middle, wrist)
        m_ring   = clac_slope(ring, wrist)
        m_pinky  = clac_slope(pinky, wrist)

        tan = [tan_angle(m_thumb, m_index), tan_angle(m_index, m_middle), tan_angle(m_middle, m_ring), tan_angle(m_ring, m_pinky), tan_angle(m_thumb, m_pinky)]

        angles = []
        for i in range(5):
            angles.append(math.degrees(math.atan(tan[i])))
        
    else:
        angles = [0.0]*5
        
    return angles

def clac_angle_Left():
    if results.left_hand_landmarks:
        wrist  = (int(results.left_hand_landmarks.landmark[0].x * w), int(results.left_hand_landmarks.landmark[0].y * h))
        thumb  = (int(results.left_hand_landmarks.landmark[4].x * w), int(results.left_hand_landmarks.landmark[4].y * h))
        index  = (int(results.left_hand_landmarks.landmark[8].x * w), int(results.left_hand_landmarks.landmark[8].y * h))
        middle = (int(results.left_hand_landmarks.landmark[12].x * w), int(results.left_hand_landmarks.landmark[12].y * h))
        ring   = (int(results.left_hand_landmarks.landmark[16].x * w), int(results.left_hand_landmarks.landmark[16].y * h))
        pinky  = (int(results.left_hand_landmarks.landmark[20].x * w), int(results.left_hand_landmarks.landmark[20].y * h))

        m_thumb  = clac_slope(thumb, wrist)
        m_index  = clac_slope(index, wrist)
        m_middle = clac_slope(middle, wrist)
        m_ring   = clac_slope(ring, wrist)
        m_pinky  = clac_slope(pinky, wrist)

        tan = [tan_angle(m_thumb, m_index), tan_angle(m_index, m_middle), tan_angle(m_middle, m_ring), tan_angle(m_ring, m_pinky), tan_angle(m_thumb, m_pinky)]

        angles = []
        for i in range(5):
            angles.append(math.degrees(math.atan(tan[i])))
            
    else:
        angles = [0.0]*5
        
    return angles

In [8]:
def sound_OUT(sound, temp):
    try:
        if temp == 'Arabic':
            lang = 'ar'
        elif temp== 'English':
            lang = 'en'
        tts = gTTS(sound, lang = lang)
        tts.save('sounds.mp3')
        playsound(r'sounds.mp3')
        os.remove(r'sounds.mp3')
    except:
        print('No sound')

In [1]:
model = arabic_words_model
actions = arabic_words

start = 0
temp = 'Arabic'

alpha = []
video = []
sentence = []

cap = cv2.VideoCapture(0)
pTime = 0
with mp_holistic.Holistic(model_complexity=0) as holistic:
    while True:
        _, frame = cap.read()
        h, w, _ = frame.shape

        frame, results = mediapipe_detection(frame, holistic)
        draw_landmarks(frame, results)

        #################################################
        if start == 0:
            if results.right_hand_landmarks or results.left_hand_landmarks:

                keypoints = extract_keypoints(results)
                keypoints += clac_angle_Right()
                keypoints += clac_angle_Left()
                keypoints += list(getDistanceFromWrist(frame, results))
                keypoints += list(getDistanceFromEar(frame, results))

                video.append(keypoints)
                video = video[-30:]
                

                if len(video) == 30:
                    res = model.predict(np.expand_dims(video, axis=0))[0]
#                     video = np.float32(video)
#                     model.set_tensor(input_details[0]['index'], np.expand_dims(video, axis=0))
#                     model.invoke()
#                     res = model.get_tensor(output_details[0]['index'])

                    if len(sentence) > 0:
                        sentence.insert(0, actions[np.argmax(res)])

                        if sentence[0] != sentence[1]:

                            if sentence[0] == 'ارقام' or sentence[0] == 'numbers':
                                if len(alpha) > 0:
                                    sound_OUT(''.join(str(e) for e in alpha[1:]), temp)
                                alpha = []
                                print('Using Numbers Model')
                                if sentence[0] == 'ارقام':
                                    print('استخدام الارقام')
                                    model = arabic_numbers_model
                                    actions = arabic_numbers
#                                 elif sentence[0] == 'numbers':
#                                     print('using numbers')
#                                     model = english_numbers_model
#                                     actions = english_numbers
                            elif sentence[0] == 'حروف' or sentence[0] == 'alphabet':
                                if sentence[0] == 'حروف':
                                    print('استخدام الحروف')
                                    model = arabic_alphabet_model
                                    actions = arabic_alphabet
                                elif sentence[0] == 'alphabet':
                                    print('Using Alphabet Model')
                                    model = english_alphabet_model
                                    actions = english_alphabet
                            elif sentence[0] == 'كلمات' or sentence[0] == 'words':
                                if len(alpha) > 0:
                                    sound_OUT(''.join(str(e) for e in alpha[1:]), temp)
                                alpha = []
                                if sentence[0] == 'كلمات':
                                    print('استخدام الكلمات')
                                    model = arabic_words_model
                                    actions = arabic_words
                                elif sentence[0] == 'words':
                                    print('using words')
                                    model = english_words_model
                                    actions = english_words
                            else:
                                print(sentence[0])
                            video = []
                            cv2.waitKey(2000)

                            if model == arabic_alphabet_model or model == english_alphabet_model:
                                if sentence[0] == 'مسافة' or sentence[0] == 'space':
                                    alpha.append(' ')
                                else:
                                    alpha.append(sentence[0])
                            else:
                                if sentence[0] not in ['حروف', 'كلمات', 'ارقام', 'numbers', 'words', 'alphabet']:
                                    sound_OUT(sentence[0], temp)
                            video = []
                        elif sentence[0] == sentence[1]:
                            video = []

                    else:
                        sentence.append(actions[np.argmax(res)])
                        print(sentence[0])
                        sound_OUT(sentence[0], temp)
                        video = []
        #################################################
        
        # FLip frame
        frame = cv2.flip(frame,1) 
            
        # Video Length
        cv2.putText(frame, f'video length {len(video)}', (15,50),\
                    cv2.FONT_ITALIC, 0.5, (0,0,255), 2)

        # Display sentence
        cv2.rectangle(frame, (0,h-60), (w, h), (255, 255, 255), -1)
        try:
            text = sentence[0]
            width = w//2-len(text)-10            
            reshaped_text = arabic_reshaper.reshape(text)
            bidi_text = get_display(reshaped_text)
            fontpath = "arial.ttf" # <== https://www.freefontspro.com/14454/arial.ttf
            font = ImageFont.truetype(fontpath, 32)
            img_pil = Image.fromarray(frame)
            draw = ImageDraw.Draw(img_pil)
            draw.text((width,h-50), bidi_text, font = font, fill = (0, 0, 0, 0))
            frame = np.array(img_pil)
        except:
            pass
        
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
#         cv2.putText(frame, f'FPS: {int(fps)}', (500,70), cv2.cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
#         frame = cv2.flip(frame,1)
        cv2.imshow('Esm3ni', frame)
        key = cv2.waitKey(1)
        if key == 27:
            break
        elif key & 0xFF == ord('r'):
            video = []
        elif key & 0xFF == ord('a'):
            video = []
            temp = 'Arabic'
            if(len(sentence) == 0):
                sentence.append('اللغة العربية')
            else:
                sentence[0] = 'اللغة العربية'
            print(sentence[0])
            sound_OUT(sentence[0], temp)
            model = arabic_words_model
            actions = arabic_words
        elif key & 0xFF == ord('e'):
            video = []
            temp = 'English'
            if(len(sentence) == 0):
                sentence.append('English')
            else:
                sentence[0] = 'English'
            print(sentence[0])
            sound_OUT(sentence[0], temp)
            model = english_words_model
            actions = english_words

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'arabic_words_model' is not defined